In [12]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
from statsmodels.api import OLS
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from ISLP.models import Stepwise, sklearn_selected, sklearn_selection_path
from functools import partial
from l0bnb import fit_path
from ISLP import load_data

In [2]:
pip install ISLP


Note: you may need to restart the kernel to use updated packages.


In [6]:
Hitters = pd.read_csv(r"C:\Users\user\Desktop\Data_Mining_Docs\ALL_CSV_FILES\Hitters.csv")
Hitters.columns




Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League', 'Division',
       'PutOuts', 'Assists', 'Errors', 'Salary', 'NewLeague'],
      dtype='object')

In [11]:
Hitters = Hitters.dropna()
Y = np.array(Hitters['Salary'])
design = MS(Hitters.columns.drop('Salary')).fit(Hitters)

In [13]:

# 2. One-hot encode categorical variables
Hitters_encoded = pd.get_dummies(Hitters, drop_first=True)

# 3. Separate response and features
Y = Hitters_encoded['Salary'].astype(float).values
X = Hitters_encoded.drop(columns='Salary')

# 4. Ensure all X values are float
X = X.astype(float).values

# 5. Estimate residual variance (sigma^2)
sigma2 = OLS(Y, X).fit().scale
print("sigma² =", sigma2)

sigma² = 100500.82193750184


In [14]:
# --- Subset Selection ---
def nCp(sigma2, estimator, X, Y):
    n, p = X.shape
    Yhat = estimator.predict(X)
    RSS = np.sum((Y - Yhat) ** 2)
    return -(RSS + 2 * p * sigma2) / n


In [15]:
neg_Cp = partial(nCp, sigma2)
strategy = Stepwise.first_peak(design, direction='forward', max_terms=len(design.terms))
hitters_MSE = sklearn_selected(OLS, strategy)
hitters_MSE.fit(Hitters, Y)
hitters_Cp = sklearn_selected(OLS, strategy, scoring=neg_Cp)
hitters_Cp.fit(Hitters, Y)


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).